In [ ]:
import numpy as np
import timeit
import networkx as nx
import importlib.util
import matplotlib.pyplot as plt


import numpy.linalg as la
import scipy


## Modificacion del codigo en busqueda de parelelizar

### Primera Idea 
Consiste en reiniciar R en cada iteracion (sin memoria durante la iteracion), y calcular cada fila de X de forma independiente utilizando la misma R y X. 
En cada iteracion se calcula $R = X^T . X$

In [ ]:
def coordinate_descent_MODIFICADO_1(A,d, max_iteraciones = None, X=None,tol=1e-5, ):
    n=A.shape[0]
    M = np.ones(n) - np.eye(n)
    if X is None:
        X = np.random.rand(n,d)
    else:
        X = X.copy()
        
    errores = []

    fold = -1
    iter = 0;
    while ( (max_iteraciones is None or iter < max_iteraciones) and abs((fold - cost_function(A, X, M))/fold) >tol):
        
        # Reinicio la R en cada iteracion, como si el metodo comenzara desde X
        R = X.T@X
        print("Iteracion: " + str(iter) + " | Costo: " + str(fold) + " | norma R: " + str(la.norm(R)))

        X_s = X.copy()

        fold = cost_function(A, X, M)
        
        for i in range(n):
            #Cada R (en este caso R2) comienza en cada iteracion desde el inicio
            R2 = R.copy()
            k=X[i,:][np.newaxis] 
            R2 -= k.T@k
            X_s[i,:] = solve_linear_system(R2,(A[i,:]@X).T,X[i,:])
            # No vuelvo a tocar R, ya que no se como hacer el merge y en este metodo lo recalculo

        X = X_s.copy()
        errores = errores + [abs((fold - cost_function(A, X, M))/fold)]
        iter += 1
    return (X, errores)

In [ ]:
nt=np.sum(n)
g = nx.stochastic_block_model(n,p1)
A = nx.to_numpy_array(g)
M = np.ones(nt) - np.eye(nt)

print("Running Coordinate descent MODIFICACION 1")
start_cd = timeit.default_timer()
(X_cd_S, errores_M1) = coordinate_descent_MODIFICADO_1(A,d, 50)
stop_cd = timeit.default_timer()
time_cd = round(stop_cd-start_cd,3)
print("Done in ",time_cd," seconds.\n")

print("Coordinate descent MODIFICADO 1   ",time_cd,"\t\t", cost_function(A,X_cd_S,M))
plt.plot(errores_M1)


Como se puede observar, no se puede ejecutar solve_linear_system(R,(A[i,:]@X).T,X[i,:]), matriz mal condicionada.

Luego de la reunion con Ernesto el dia Jueves 18/4 se propuso otra idea, de todas formas insito aun en la primera a ver si podemos conseguir resultados.

### Segunda idea

Intentemos lo mismo que en la idea original, pero actualizando R recien al final de cada iteracion. Donde actualizo R una vez que tengo la nueva iteracion de X calculado

In [ ]:
def coordinate_descent_MODIFICADO_2(A,d,max_iteraciones,X=None,tol=1e-5):
    n=A.shape[0]
    M = np.ones(n) - np.eye(n)
    if X is None:
        X = np.random.rand(n,d)
    else:
        X = X.copy()
    
    
    errores = []

    R = X.T@X 

    fold = -1
    iter = 0
    
    while ((max_iteraciones is None or iter < max_iteraciones) and abs((fold - cost_function(A, X, M))/fold) >tol):

        print(iter)
        X_s = X.copy()
        fold = cost_function(A, X, M)
        print("Iteracion: " + str(iter) + " | Costo: " + str(fold) + " | norma R: " + str(la.norm(R)))
        for i in range(n):
            R2 = R.copy()
            k=X[i,:][np.newaxis] #(*1)
            R2 -= k.T@k
            X_s[i,:] = solve_linear_system(R2,(A[i,:]@X).T,X[i,:])

        
        # Vuelvo a aplicar los cambios de la linea (*1), pero utilizando la iteracion anterior
        for i in range(n):
            k=X[i,:][np.newaxis]
            R -= k.T@k

        X = X_s.copy()

        # Aplico cambios R pero ahora con la X totalmente calculada en el paso anterior
        for i in range(n):
            k=X[i,:][np.newaxis]
            R += k.T@k       
        
        # Calculo el error relativo
        errores = errores + [abs((fold - cost_function(A, X, M))/fold)]

        iter+= 1

    return (X, errores)

In [ ]:
nt=np.sum(n)
g = nx.stochastic_block_model(n,p1)
A = nx.to_numpy_array(g)
M = np.ones(nt) - np.eye(nt)

print("Running Coordinate descent Modificado 2")
start_cd = timeit.default_timer()
(X_cd_S, errores) = coordinate_descent_MODIFICADO_2(A,d,50)
stop_cd = timeit.default_timer()
time_cd = round(stop_cd-start_cd,3)
print("Done in ",time_cd," seconds.\n")

print("Coordinate descent MODIFICADO 2   ",time_cd,"\t\t", cost_function(A,X_cd_S,M))
plt.plot(errores)
